In [17]:
import pandas as pd
import numpy as np
import re
from functools import reduce

In [3]:
df = pd.read_csv('/opt/covidcg_flu/data_flu_genbank/metadata.csv')
df.head()

/opt/miniconda3/envs/covid-cg/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3397: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Accession ID,database,isolate_id,genus,serotype,virus_name,region,country,collection_date,submission_date,...,isolation_source,biosample_accession,authors,publications,n_subtype,segment,division,location,length,percent_ambiguous
0,NC_002016,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1982-06-09,...,Unknown,Unknown,"Winter,G., Fields,S.",6927841,1,7,-1,-1,1027,0.0
1,NC_002022,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1982-11-03,...,Unknown,Unknown,"Fields,S., Winter,G.",7060132,1,3,-1,-1,2233,0.0
2,NC_002023,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1982-11-03,...,Unknown,Unknown,"Fields,S., Winter,G.",7060132,1,1,-1,-1,2341,0.0
3,NC_002017,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1983-07-13,...,Unknown,Unknown,"Winter,G., Fields,S., Brownlee,G.G.",7278968,1,4,-1,-1,1778,0.0
4,NC_002020,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1993-08-02,...,Unknown,Unknown,"Hall,R.M., Air,G.M., Winter,G., Fields,S., Gai...","7241645, 7208353, 7465426, 7385583",1,8,-1,-1,890,0.0


In [8]:
df['location'].value_counts()

-1             985747
-1              26253
NY               8857
King County      7608
New York         6636
                ...  
Columbia            1
Dallas              1
White Rock          1
Lake Towada         1
Village             1
Name: location, Length: 442, dtype: int64

In [9]:
df['division'].value_counts()

-1                 304969
Michigan            54267
California          36842
Washington          23547
Minnesota           21613
                    ...  
Ribeirao Preto          1
Alwadi Algidid          1
Kandal province         1
Benisuif                1
WEST BENGAL             1
Name: division, Length: 3108, dtype: int64

In [11]:
df['country'].value_counts()

USA                 725641
China                88370
Australia            17307
Viet Nam             13583
Japan                12946
                     ...  
Jamaica                  1
Suriname                 1
Albania                  1
Montenegro               1
French Polynesia         1
Name: country, Length: 163, dtype: int64

In [12]:
df['region'].value_counts()

North America    755916
Asia             195933
Europe            62536
Oceania           22811
Africa            19934
South America     17498
Antarctica          201
Name: region, dtype: int64

In [57]:
def clean_location_data(location_df, location_corretions):
    """Fix typos, unify nomenclature in location data
    """

    # Load rules
    location_correction_df = pd.read_csv(location_corretions, comment="#")
    # region_pattern,country_pattern,division_pattern,location_pattern,out_region,out_country,out_division,out_location,comment

    for i, rule in location_correction_df.iterrows():
        if i % 100 == 0:
            print(f'running location cleaning rule {i}/{len(location_correction_df)}')
        # print(rule)
        input_rule = {
            "region": rule["region_pattern"],
            "country": rule["country_pattern"],
            "division": rule["division_pattern"],
            "location": rule["location_pattern"],
        }
        output_rule = {
            "region": rule["out_region"],
            "country": rule["out_country"],
            "division": rule["out_division"],
            "location": rule["out_location"],
        }

        # Get matching entries for the input rule
        # by creating a logical mask
        # Start out with matching everything
        loc_mask = pd.Series(np.repeat(True, len(location_df)))
        for key in input_rule.keys():
            if type(input_rule[key]) is not str or not input_rule[key]:
                continue

            vals = input_rule[key].split("|")
            # Make it a list if it's just a single value
            if type(vals) is not list:
                vals = [vals]
            vals = [str(val) for val in vals]

            # Turn each value into a logical mask
            vals = [location_df[key] == v for v in vals]
            # Combine logical masks with logical ORs, and merge into the master mask with AND
            loc_mask = loc_mask & reduce(lambda x, y: (x | y), vals)

        # Set the output rules on the matching entries from loc_mask
        for out_key in output_rule.keys():
            if (
                type(output_rule[out_key]) is not str
                and type(output_rule[out_key]) is not int
            ):
                continue
            location_df.loc[loc_mask, out_key] = output_rule[out_key]

    # Done
    return location_df

In [14]:
location_corrections = pd.read_csv('/opt/covidcg_flu/static_data/flu/location_corrections.csv')
location_corrections

,region_pattern,country_pattern,division_pattern,location_pattern,out_region,out_country,out_division,out_location,comment
0,NaN,NaN,Unknown|unknown,NaN,NaN,NaN,-1,NaN,Unset unknown divisions
1,NaN,NaN,NaN,Unknown|unknown,NaN,NaN,NaN,-1,Unset unknown locations
2,NaN,CotedIvoire|Cote dIvoire|Côte d'Ivoire,NaN,NaN,NaN,Côte D'Ivoire,NaN,NaN,NaN
3,NaN,Democratic Republic of the Congo,NaN,NaN,NaN,DRC,NaN,NaN,Abbreviate name to save space
4,NaN,DRC,Kongo Central,NaN,NaN,NaN,Kongo-Central,NaN,Fix typos
...,...,...,...,...,...,...,...,...,...
904,NaN,USA,VA,NaN,NaN,NaN,Virginia,NaN,NaN
905,NaN,USA,WA,NaN,NaN,NaN,Washington,NaN,NaN
906,NaN,USA,WV,NaN,NaN,NaN,West Virginia,NaN,NaN
907,NaN,USA,WI,NaN,NaN,NaN,Wisconsin,NaN,NaN


In [15]:
# general strategy
# first clean up the states and merge 2 letter abbreviations with the full state names
# then detect counties mislabeled as states and flip the location and division cols
# then re-run the state cleanup
location_df = df[['region', 'country', 'division', 'location']].copy()
location_df

,region,country,division,location
0,North America,USA,-1,-1
1,North America,USA,-1,-1
2,North America,USA,-1,-1
3,North America,USA,-1,-1
4,North America,USA,-1,-1
...,...,...,...,...
1074824,North America,USA,Ohio,-1
1074825,North America,USA,Iowa,-1
1074826,North America,USA,Iowa,-1
1074827,North America,USA,Iowa,-1


In [19]:
a = clean_location_data(location_df, '/opt/covidcg_flu/static_data/flu/location_corrections.csv')
a


,region,country,division,location
0,North America,USA,-1,-1
1,North America,USA,-1,-1
2,North America,USA,-1,-1
3,North America,USA,-1,-1
4,North America,USA,-1,-1
...,...,...,...,...
1074824,North America,USA,Ohio,-1
1074825,North America,USA,Iowa,-1
1074826,North America,USA,Iowa,-1
1074827,North America,USA,Iowa,-1


In [21]:
a['location'].value_counts()

-1                        987007
-1                         24935
NY                          7817
King County                 7608
New York                    5902
                           ...  
Luang Prabang District         1
Guizhou                        1
Florianopolis                  1
Lishui                         1
Behera                         1
Name: location, Length: 463, dtype: int64

In [31]:
a['division'].value_counts().index.values.tolist()

['-1',
 'California',
 'Michigan',
 'Minnesota',
 'Washington',
 'Texas',
 'Colorado',
 'Iowa',
 'Massachusetts',
 'Florida',
 'New York',
 'Wisconsin',
 'Ohio',
 'Pennsylvania',
 'Maryland',
 'Illinois',
 'Idaho',
 'North Carolina',
 'South Dakota',
 'Hawaii',
 'Utah',
 'New Jersey',
 'Missouri',
 'Arizona',
 'Indiana',
 'Delaware',
 'Jiangxi',
 'Alaska',
 'North Dakota',
 'Rhode Island',
 'Nevada',
 'Georgia',
 'Oregon',
 'Boston',
 'Virginia',
 'Nebraska',
 'Montana',
 'Connecticut',
 'Tennessee',
 'Shanghai',
 'New Mexico',
 'Basel-Stadt',
 'Managua',
 'Hunan',
 'New Hampshire',
 'Mississippi',
 'Oklahoma',
 'Washington DC',
 'Louisiana',
 'West Virginia',
 'Kentucky',
 'Guangdong',
 'South Carolina',
 'Arkansas',
 'Sydney',
 'Maine',
 'Kansas',
 'Wyoming',
 'Victoria',
 'Anhui',
 'Fujian',
 'Vermont',
 'Rhineland-Palatinate',
 'Shanxi',
 'Canterbury',
 'Alabama',
 'Ontario',
 'South Australia',
 'Manitoba',
 'Brisbane',
 'Taipei',
 'SE Wisconsin',
 'Hong Kong',
 'Alberta',
 'Henan

In [93]:
state_abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
state_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut",
    "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana",
    "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
    "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska",
    "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York",
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming", "Washington DC"
]

# a['location'].isin(state_abbreviations + state_names).sum()

# a['division'].str.match(r'.*county$', case=False).sum()
# a['division'].str.match(r'.*province$', case=False).sum()

# Flip division and location for entries where division ends with "county" or "province", 
# or for entries where division is a state abbreviation
flip_mask = (
    a['division'].str.match(r'.*county$', case=False) | 
    a['division'].str.match(r'.*province$', case=False) |
    a['location'].isin(state_abbreviations + state_names + [n + ' state' for n in state_names])
)
flip_mask.sum()

232

In [80]:
a.loc[flip_mask, ['division', 'location']] = a.loc[flip_mask, ['location', 'division']].values
a = clean_location_data(a, '/opt/covidcg_flu/static_data/flu/location_corrections.csv')
a

running location cleaning rule 0/1005
running location cleaning rule 100/1005
running location cleaning rule 200/1005
running location cleaning rule 300/1005
running location cleaning rule 400/1005
running location cleaning rule 500/1005
running location cleaning rule 600/1005
running location cleaning rule 700/1005
running location cleaning rule 800/1005
running location cleaning rule 900/1005
running location cleaning rule 1000/1005


,region,country,division,location
0,North America,USA,-1,-1
1,North America,USA,-1,-1
2,North America,USA,-1,-1
3,North America,USA,-1,-1
4,North America,USA,-1,-1
...,...,...,...,...
1074824,North America,USA,Ohio,-1
1074825,North America,USA,Iowa,-1
1074826,North America,USA,Iowa,-1
1074827,North America,USA,Iowa,-1


In [81]:
a['division'].value_counts()

-1                 305704
California          60740
Michigan            56952
New York            33182
Minnesota           26927
                    ...  
Elwadi-elgaded          1
Kafr-Elshekh            1
Ein Gedi                1
South Carrolina         1
WEST BENGAL             1
Name: division, Length: 2470, dtype: int64

In [82]:
a['division'].value_counts().index.values.tolist()

['-1',
 'California',
 'Michigan',
 'New York',
 'Minnesota',
 'Massachusetts',
 'Washington',
 'Texas',
 'Iowa',
 'Colorado',
 'Florida',
 'Wisconsin',
 'Ohio',
 'Pennsylvania',
 'Illinois',
 'Maryland',
 'North Carolina',
 'Idaho',
 'South Dakota',
 'Hawaii',
 'New Jersey',
 'Utah',
 'Missouri',
 'Arizona',
 'Indiana',
 'Delaware',
 'Tennessee',
 'Jiangxi',
 'Alaska',
 'North Dakota',
 'Oregon',
 'Georgia',
 'Rhode Island',
 'Nevada',
 'Virginia',
 'Nebraska',
 'Montana',
 'Connecticut',
 'Shanghai',
 'New Mexico',
 'Basel-Stadt',
 'Managua',
 'Kentucky',
 'Louisiana',
 'New Hampshire',
 'Hunan',
 'Mississippi',
 'Oklahoma',
 'Washington DC',
 'West Virginia',
 'Guangdong',
 'South Carolina',
 'Kansas',
 'Arkansas',
 'Sydney',
 'Maine',
 'Wyoming',
 'Victoria',
 'Anhui',
 'Vermont',
 'Fujian',
 'Rhineland-Palatinate',
 'Alabama',
 'Shanxi',
 'Canterbury',
 'Ontario',
 'South Australia',
 'Manitoba',
 'Brisbane',
 'Taipei',
 'Hong Kong',
 'Alberta',
 'Henan',
 'England',
 'Shandong',


In [83]:
a.loc[a['country'] == 'USA', 'division'].value_counts().head(100)

-1                                      123032
California                               60740
Michigan                                 56952
New York                                 33182
Minnesota                                26927
Massachusetts                            26112
Washington                               25607
Texas                                    24255
Iowa                                     20538
Colorado                                 20242
Florida                                  19475
Wisconsin                                18686
Ohio                                     13570
Pennsylvania                             11079
Illinois                                 10326
Maryland                                 10190
North Carolina                            9647
Idaho                                     9552
South Dakota                              9281
Hawaii                                    9105
New Jersey                                9063
Utah         

In [84]:
a.loc[a['country'] == 'USA', 'division'].value_counts().index.values.tolist()

['-1',
 'California',
 'Michigan',
 'New York',
 'Minnesota',
 'Massachusetts',
 'Washington',
 'Texas',
 'Iowa',
 'Colorado',
 'Florida',
 'Wisconsin',
 'Ohio',
 'Pennsylvania',
 'Illinois',
 'Maryland',
 'North Carolina',
 'Idaho',
 'South Dakota',
 'Hawaii',
 'New Jersey',
 'Utah',
 'Missouri',
 'Arizona',
 'Indiana',
 'Delaware',
 'Tennessee',
 'Alaska',
 'North Dakota',
 'Oregon',
 'Georgia',
 'Rhode Island',
 'Nevada',
 'Virginia',
 'Nebraska',
 'Montana',
 'Connecticut',
 'New Mexico',
 'Kentucky',
 'Louisiana',
 'New Hampshire',
 'Mississippi',
 'Oklahoma',
 'Washington DC',
 'West Virginia',
 'South Carolina',
 'Kansas',
 'Arkansas',
 'Maine',
 'Wyoming',
 'Vermont',
 'Alabama',
 'Brownsville',
 'Mispillion Harbor',
 'Minto Flats',
 "Cook's Beach",
 "Reed's Beach",
 'Santa Clara',
 "South Reed's Beach",
 'San Juan',
 'University of Vermont Medical Center',
 'Thief Lake',
 'Agassiz NWR',
 'PR',
 'Yigo',
 'Lake Co.',
 'DeSoto',
 'Guam',
 'Ft Carson',
 'San Francisco',
 'Lake Aud

In [95]:
((a['country'] == 'USA') & (~a['division'].isin(state_names)) & ~(a['division'] == "-1")).sum()

1400

In [99]:
a.loc[((a['country'] == 'USA') & (~a['division'].isin(state_names)) & ~(a['division'] == "-1"))]['location'].value_counts()

-1                    770
Cape May county       160
Sussex county         128
Interior Alaska        72
Burlington             40
Farmes Pool            32
Marshall County        24
WI east                24
Co                     20
McLean county          16
Glenn County           16
Middlesex county       16
Marshall county        16
NW                      8
Roseau county           8
Cumberland county       8
Kent county             8
Dunn county             8
Chambers County TX      8
Baltimore County        8
Yaphank                 4
Petersburg              3
Johnston                2
Columbia                1
Name: location, dtype: int64

In [55]:
a.loc[a['division'] == 'Boston', 'location'].value_counts()

-1    1040
Name: location, dtype: int64

In [64]:
a['location'].value_counts()

-1                    996278
-1                     24935
King County             7609
Boston                  6174
Los Angeles County      2012
                       ...  
Hallam                     1
Dornogobi province         1
Kandal province            1
Lake Ogawara               1
Dc                         1
Name: location, Length: 748, dtype: int64

In [100]:
df

,Accession ID,database,isolate_id,genus,serotype,virus_name,region,country,collection_date,submission_date,...,isolation_source,biosample_accession,authors,publications,n_subtype,segment,division,location,length,percent_ambiguous
0,NC_002016,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1982-06-09,...,Unknown,Unknown,"Winter,G., Fields,S.",6927841,1,7,-1,-1,1027,0.0
1,NC_002022,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1982-11-03,...,Unknown,Unknown,"Fields,S., Winter,G.",7060132,1,3,-1,-1,2233,0.0
2,NC_002023,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1982-11-03,...,Unknown,Unknown,"Fields,S., Winter,G.",7060132,1,1,-1,-1,2341,0.0
3,NC_002017,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1983-07-13,...,Unknown,Unknown,"Winter,G., Fields,S., Brownlee,G.G.",7278968,1,4,-1,-1,1778,0.0
4,NC_002020,RefSeq,GCF_000865725.1,Alphainfluenzavirus,H1N1,A/Puerto-Rico/8/1934,North America,USA,1934,1993-08-02,...,Unknown,Unknown,"Hall,R.M., Air,G.M., Winter,G., Fields,S., Gai...","7241645, 7208353, 7465426, 7385583",1,8,-1,-1,890,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074824,PV901735,GenBank,A/swine/Ohio/A02858688/2025,Alphainfluenzavirus,H3N2,A/swine/Ohio/A02858688/2025,North America,USA,2025-06-18,2025-07-08,...,lung,Unknown,"USDA Swine Surveillance,A., Lin,K., Dohrn,C.",Unknown,2,6,Ohio,-1,1410,0.0
1074825,PV901738,GenBank,A/swine/Iowa/A02858690/2025,Alphainfluenzavirus,H1N1,A/swine/Iowa/A02858690/2025,North America,USA,2025-06-19,2025-07-08,...,lung,Unknown,"USDA Swine Surveillance,A., Lin,K., Dohrn,C.",Unknown,1,4,Iowa,-1,1701,0.0
1074826,PV901739,GenBank,A/swine/Iowa/A02858690/2025,Alphainfluenzavirus,H1N1,A/swine/Iowa/A02858690/2025,North America,USA,2025-06-19,2025-07-08,...,lung,Unknown,"USDA Swine Surveillance,A., Lin,K., Dohrn,C.",Unknown,1,6,Iowa,-1,1410,0.0
1074827,PV901740,GenBank,A/swine/Iowa/A02858691/2025,Alphainfluenzavirus,H1N1,A/swine/Iowa/A02858691/2025,North America,USA,2025-06-19,2025-07-08,...,lung,Unknown,"USDA Swine Surveillance,A., Lin,K., Dohrn,C.",Unknown,1,4,Iowa,-1,1701,0.0


In [101]:
a

,region,country,division,location
0,North America,USA,-1,-1
1,North America,USA,-1,-1
2,North America,USA,-1,-1
3,North America,USA,-1,-1
4,North America,USA,-1,-1
...,...,...,...,...
1074824,North America,USA,Ohio,-1
1074825,North America,USA,Iowa,-1
1074826,North America,USA,Iowa,-1
1074827,North America,USA,Iowa,-1


In [103]:
df.loc[:, ['region', 'country', 'division', 'location']] = a